## Taller - MapReduce

Si está trabajando en Google Colaboratory, puede instalar la librería con `!pip install mrjob`.

### Ejercicio 1: Conteo de caracteres

Completa la clase `CharCount` implementando un programa map-reduce que, dado un texto de entrada, cuente:

- El número de veces que aparece cada caracter (la clase `Counter` de Python puede ser de utilidad).
    - Los espacios no deben aparecer en el conteo.
    - 'A' y 'a' se deben tratar como un solo caracter. 
- El número total de vocales.
- El número total de consonantes.

y emita como resultado tres tuplas como en el ejemplo siguiente.

_Ejemplo de ejecución_

    Ancient influences have helped spawn variant interpretations 
    of the nature of history which have evolved over the centuries 
    and continue to change today. The modern study of history is 
    wide-ranging, and includes the study of specific regions and 
    the study of certain topical or thematical elements of 
    historical investigation. Often history is taught as part of 
    primary and secondary education, and the academic study of 
    history is a major discipline in University studies
    
_Resultado_

    "a"	32
    "c"	18
    "d"	20
    ...
    "y"	12
    "vowels" 153
    "consonants" 247

<font style="color: red">La función map puede devolver varios pares *(llave,valor)* utilizando varias veces la instrucción `yield`

In [ ]:
%%writefile CharCount.py
from mrjob.job import MRJob
from collections import Counter

class CharCount(MRJob):

    def mapper(self, _, line):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"

    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    CharCount.run()

In [ ]:
%%script python CharCount.py --quiet
Ancient influences have helped spawn variant interpretations 
of the nature of history which have evolved over the centuries 
and continue to change today. The modern study of history is 
wide-ranging, and includes the study of specific regions and 
the study of certain topical or thematical elements of 
historical investigation. Often history is taught as part of 
primary and secondary education, and the academic study of 
history is a major discipline in University studies

### Ejercicio 2: Suma de Matrices

Representamos dos matrices **a** y **b** de 2x2 de la siguiente forma:

    ["a", 0, 0, 32]
    ["a", 0, 1, 69]
    ["a", 1, 0, 18]
    ["a", 1, 1, 28]
    ["b", 0, 0, 18]
    ["b", 0, 1, 69]
    ["b", 1, 0, 28]
    ["b", 1, 1, 32]

es decir

    a = [32 69]     b = [18 69]    a + b = [50 138]
        [18 28]         [28 32]            [46  60]
        
        
tienes que implementar un programa map-reduce que acepte como entrada las dos matrices y realice la suma

In [ ]:
%%writefile MatrixSum.py
from mrjob.job import MRJob
import json

class MatrixSum(MRJob):

    def mapper(self, _, line):
        _, x, y, val = json.loads(line)
        # TU CODIGO AQUI
        yield "KEY", "VALUE"

    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    MatrixSum.run()

In [ ]:
%%script python MatrixSum.py -q 
["a", 0, 0, 32]
["a", 0, 1, 69]
["a", 1, 0, 18]
["a", 1, 1, 28]
["b", 0, 0, 18]
["b", 0, 1, 69]
["b", 1, 0, 28]
["b", 1, 1, 32]

### Ejercicio 3: Matriz dispersa

Representamos dos matrices dispersas **a** y **b** de 2x2 de la siguiente forma:

    ["a", 0, 0, 63]
    ["a", 1, 1, 32]
    ["b", 0, 0, 69]
    ["b", 0, 1, 18]
    ["b", 1, 1, 28]

es decir

    a = [63  0]     b = [69 18]    a x b = [4347 1134]
        [ 0 32]         [ 0 28]            [   0  896]
        
        
tienes que implementar un programa map-reduce que acepte como entrada las matrices en el formato disperso y devuelva la multiplicación de ambas matrices en formato también disperso.

_Ejemplo de ejecución_ (entrada es en formato **json**)

    ["a", 0, 0, 63]
    ["a", 1, 1, 32]
    ["b", 0, 0, 69]
    ["b", 0, 1, 18]
    ["b", 1, 1, 28]

_Resultado_

    [0, 0]	4347
    [0, 1]	1134
    [1, 1]	896

**Ayuda: llaves en la función map, entradas reduce**

map:
Usando llaves de tres posiciones, las primeras dos indican la posición en la matriz final y la tercera posición está asociada a la posición en la suma.

La posición [0,0] en la matriz final se calcula con a[0,0]*b[0,0] + a[0,1]*b[1,0]. Donde _a[0,0]*b[0,0]_ lo podríamos ver como la primera posición de la suma y _a[0,1]*b[1,0]_ como la segunda.
    
    entrada reducer
    key  values 
    000, [63, 69]
    010, [63, 18]
    011, [28]
    100, [69]
    101, [32]
    110, [18]
    111, [32, 28]    
Una alternativa es utilizar dos tareas reduce, una en la que se realicen las multiplicaciones y una posterior en la que se sumen los dos valores por cada celda. Al tratarse de matrices dispersas, se debe tener en cuenta que si faltan posiciones en la matriz, se asume que el valor en esa posición es cero (0). <font style="color: red"> Lo último debe tenerse en cuenta a la hora de multiplicar

In [ ]:
%%writefile Matmult.py
from mrjob.job import MRJob
import json

N=2 # dimension de las matrices cuadradas a multiplicar

class Matmult(MRJob):

    def mapper(self, _, line):
        id, y, x, val = json.loads(line)
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    Matmult.run()

In [ ]:
%%script python Matmult.py -q
["a", 0, 0, 63]
["a", 1, 1, 32]
["b", 0, 0, 69]
["b", 0, 1, 18]
["b", 1, 1, 28]

### Ejercicio 4: Extracción de información de archivos climáticos

Considere los archivos de datos climáticos de [NDCC](https://www1.ncdc.noaa.gov/pub/data/igra/) provistos ([Data/climate-data.zip](Data/climate-data.zip)).

Implementar un algoritmo MapReduce que determine los registros *​válidos ​y ​confiables* ​que indiquen que la temperatura del aire superó los 0°C y exprese su número total en valor absoluto y su porcentaje sobre el total de registros en el archivo provisto.

Adicionalmente muestre la temperatura promedio.

Consulte el documento **igra2-data-format.txt** el cual contiene la información de cómo están organizados los datos y cuáles son válidos. De particular interés para nuestra tarea son las columnas _TEMP_ y _TFLAG_.

El resultado es de la forma:

    "temperatura promedio"	-29.309409190371984

    "registros temperatura >0"	149
    
    "registros temperatura >0 [%]"	32.60393873085339

In [ ]:
%%writefile Temp.py
from mrjob.job import MRJob

class Temp(MRJob):

    def mapper(self, _, line):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
    def reducer(self, key, values):
        # TU CODIGO AQUI
        yield "KEY", "VALUE"
        
if __name__ == '__main__':
    Temp.run()

In [ ]:
%%script python Temp.py -q climate-data/*
--